<a href="https://colab.research.google.com/github/yw228/CS452/blob/main/SparkSQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 13.1 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824025 sha256=e7f576142d042f9a2bafe479c673071a8283d6b5ee0c8174bbdd4898adfeab4f
  Stored in directory: /root/.cache/pip/wheels/b1/59/a0/a1a0624b5e865fd389919c1a10f53aec9b12195d6747710baf
Successfully built pyspark


In [26]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import sum, desc
import re
import pandas as pd

In [3]:
# Mount my Google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
spark = SparkSession.builder \
    .appName("SparkSQLExercise") \
    .getOrCreate()
sc = spark.sparkContext
sc.setLogLevel('WARN')
sc.uiWebUrl

'http://73c90f6aae2d:4040'

In [21]:
myfile = "drive/MyDrive/Colab Notebooks/cs452/inputData/us-countries.csv"

df = spark.read.csv(myfile, header=True, inferSchema=True)



In [24]:
df.show()

+-------------------+-----------+----------+-----+-----+------+
|               date|     county|     state| fips|cases|deaths|
+-------------------+-----------+----------+-----+-----+------+
|2020-01-21 00:00:00|  Snohomish|Washington|53061|    1|     0|
|2020-01-22 00:00:00|  Snohomish|Washington|53061|    1|     0|
|2020-01-23 00:00:00|  Snohomish|Washington|53061|    1|     0|
|2020-01-24 00:00:00|       Cook|  Illinois|17031|    1|     0|
|2020-01-24 00:00:00|  Snohomish|Washington|53061|    1|     0|
|2020-01-25 00:00:00|     Orange|California| 6059|    1|     0|
|2020-01-25 00:00:00|       Cook|  Illinois|17031|    1|     0|
|2020-01-25 00:00:00|  Snohomish|Washington|53061|    1|     0|
|2020-01-26 00:00:00|   Maricopa|   Arizona| 4013|    1|     0|
|2020-01-26 00:00:00|Los Angeles|California| 6037|    1|     0|
|2020-01-26 00:00:00|     Orange|California| 6059|    1|     0|
|2020-01-26 00:00:00|       Cook|  Illinois|17031|    1|     0|
|2020-01-26 00:00:00|  Snohomish|Washing

2.Write code to find the county with the most deaths

In [35]:
total_Death_df = df.groupBy("county").agg(sum("deaths").alias("total_deaths"))
#total_Death_df.show()
most_death_county = total_Death_df.orderBy(desc("total_deaths")).first()["county"]
print("The county with the most deaths is:", most_death_county)

The county with the most deaths is: New York City


3.Write code to find the county with the most cases

In [38]:
total_case_df = df.groupBy("county").agg(sum("cases").alias("total_cases"))
#total_case_df.show()
most_cases_county = total_case_df.orderBy(desc("total_cases")).first()["county"]
print("The county with the most deaths is:", most_cases_county)

The county with the most deaths is: Los Angeles


4.Write code to find the total number of deaths in Utah county

In [53]:
utah_county_death = df.filter(df.county == "Utah").agg(sum("deaths")).first()[0]

print("The total number of deaths in Utah county is:", utah_county_death)

The total number of deaths in Utah county is: 270727


5. Write code to find the death rate for each state and sort the states by death rate

In [54]:
df.show(100)

+-------------------+-------------+-------------+-----+-----+------+
|               date|       county|        state| fips|cases|deaths|
+-------------------+-------------+-------------+-----+-----+------+
|2020-01-21 00:00:00|    Snohomish|   Washington|53061|    1|     0|
|2020-01-22 00:00:00|    Snohomish|   Washington|53061|    1|     0|
|2020-01-23 00:00:00|    Snohomish|   Washington|53061|    1|     0|
|2020-01-24 00:00:00|         Cook|     Illinois|17031|    1|     0|
|2020-01-24 00:00:00|    Snohomish|   Washington|53061|    1|     0|
|2020-01-25 00:00:00|       Orange|   California| 6059|    1|     0|
|2020-01-25 00:00:00|         Cook|     Illinois|17031|    1|     0|
|2020-01-25 00:00:00|    Snohomish|   Washington|53061|    1|     0|
|2020-01-26 00:00:00|     Maricopa|      Arizona| 4013|    1|     0|
|2020-01-26 00:00:00|  Los Angeles|   California| 6037|    1|     0|
|2020-01-26 00:00:00|       Orange|   California| 6059|    1|     0|
|2020-01-26 00:00:00|         Cook

In [77]:
from pyspark.sql.functions import round
total_Death_df = df.groupBy("state").agg(((sum("deaths")/sum("cases"))*100).alias("Death rate") )
total_Death_df = total_Death_df.withColumn("Death rate", (round(total_Death_df["Death rate"], 2))).orderBy(desc("Death rate"))
total_Death_df.show() 



+--------------------+----------+
|               state|Death rate|
+--------------------+----------+
|          New Jersey|      2.46|
|            New York|      2.33|
|         Connecticut|       2.3|
|       Massachusetts|      2.22|
|        Pennsylvania|      2.05|
|         Mississippi|      1.96|
|            Michigan|      1.94|
|           Louisiana|      1.91|
|            Maryland|      1.87|
|District of Columbia|       1.8|
|             Arizona|      1.74|
|          New Mexico|      1.73|
|             Alabama|      1.72|
|             Georgia|      1.67|
|             Indiana|      1.64|
|            Illinois|      1.63|
|              Nevada|       1.6|
|                Ohio|      1.58|
|               Texas|      1.55|
|        Rhode Island|      1.54|
+--------------------+----------+
only showing top 20 rows



6.Write code to something else interesting with this data – your choice

Find the State that has the most death

In [79]:
total_Death_df = df.groupBy("state").agg(sum("deaths").alias("total_deaths"))
#total_Death_df.show()
most_death_states = total_Death_df.orderBy(desc("total_deaths")).first()["state"]
print("The states with the most deaths is:", most_death_states)

The states with the most deaths is: California
